In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from enum import Enum
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_key = ''

In [3]:
instructor.patch()

In [4]:
from typing import List

class AnswerEnum(str, Enum):
    A = 'A'
    B = 'B'
    C = 'C'
    D = 'D'
    
class Selective_QA(BaseModel):
    question: str
    A: str
    B: str
    C: str
    D: str
    answer: AnswerEnum
        
class QAS(BaseModel):
    qa: List[Selective_QA]

In [5]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo-0613',
            response_model=QAS,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.dict()
    except Exception as e:
        return None

In [6]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
paragraph ```
{l['paragraph']}
```

berdasarkan paragraph, jana soalan melayu dan jawapan melayu
"""
    s = s.strip()
    r = predict(s)
    l['qa'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [7]:
data = []

with open('paragraph-majalahsains.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data.append({'paragraph': l})
        
len(data)

3431

In [8]:
# !rm -rf qa-majalahsains
!mkdir qa-majalahsains

mkdir: cannot create directory ‘qa-majalahsains’: File exists


In [9]:
i = 0
max_worker = 1
b = data[i: i + max_worker]
filenames = [(os.path.join('qa-majalahsains', f'{i + k}.json'), l) for k, l in enumerate(b)]

In [10]:
generate(filenames[0][0], filenames[0][1])

In [11]:
from tqdm import tqdm

max_worker = 10
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('qa-majalahsains', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

 42%|████████████████▋                       | 144/344 [00:00<00:00, 678.32it/s]/tmp/ipykernel_3041766/794675768.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  return response.dict()
100%|███████████████████████████████████████| 344/344 [1:23:36<00:00, 14.58s/it]
